This notebook takes the raw data found in multiple files within the "raw" folder and places them all into one file for a given month. The data is separated into motor vehicle theft and theft and finally geocoded so that it can be used in the map construction.

In [1]:
import pandas as pd
import os
import geocoder
import requests

In [2]:
for folder in os.listdir('./raw'):
    file_list = []
    for file in os.listdir('./raw/' + folder):
        if(file.endswith('.xls')):
            data = pd.read_excel('./raw/' + folder + '/' + file)
            file_list.append(data)
   
    df = pd.concat(file_list, ignore_index=True)
    
    columns = ['Incident Number','Date', 'Time', 'Police District','Offense 1',
          'Offense 2', 'Offense 3', 'Offense 4', 'Offense 5', 'Location']
    df.columns = columns
    
    df = df[df['Offense 1'].notnull()]
    df = df[df['Location'].notnull()]
    
    df.to_csv('./raw/' + folder + 'Complete.csv', sep=';', index = False)

In [3]:
def address_to_latlong(address):
    '''Returns the latitude and longitude for a given address'''
    address += ", Milwaukee, WI"
    location = geocoder.arcgis(address, session = session)
    
    if not location.latlng:
        print("Couldn't Parse: "  + address )
    else:
        print(f'Parsed: {address} - {location.latlng}')
    return location.latlng

In [4]:
for file in os.listdir('./raw/'):
    if(file.endswith('.csv')):
        df = pd.read_csv('./raw/'+ file, sep =';')
    
        zipped = zip(df['Offense 1'].items(), df['Offense 2'].items(), 
        df['Offense 3'].items(), df['Offense 4'].items(), df['Offense 5'].items())
        keyword = 'theft'
        
        for off_1, off_2, off_3, off_4, off_5 in zipped:
            if(not keyword in off_1[1].lower() and
              (pd.isnull(off_2[1]) or not keyword in off_2[1].lower()) and
              (pd.isnull(off_3[1]) or not keyword in off_3[1].lower()) and
              (pd.isnull(off_4[1]) or not keyword in off_4[1].lower()) and
              (pd.isnull(off_5[1]) or not keyword in off_5[1].lower())):
                df = df.drop(off_1[0])
 
        df['Address'] = df['Location'].copy()
            
        theft = df.copy()
        m_theft = df.copy()
        
        zipped = zip(df['Offense 1'].items(), df['Offense 2'].items(), 
        df['Offense 3'].items(), df['Offense 4'].items(), df['Offense 5'].items())
        keyword = 'motor vehicle theft'
        
        for off_1, off_2, off_3, off_4, off_5 in zipped:
            if(not keyword in off_1[1].lower() and
                (pd.isnull(off_2[1]) or not keyword in off_2[1].lower()) and
                (pd.isnull(off_3[1]) or not keyword in off_3[1].lower()) and
                (pd.isnull(off_4[1]) or not keyword in off_4[1].lower()) and
                (pd.isnull(off_5[1]) or not keyword in off_5[1].lower())):
                m_theft = m_theft.drop(off_1[0])
            else:
                theft = theft.drop(off_1[0])
                
        
        with requests.Session() as session:
            # This is likely to be very slow
            # depending on the geocoding service used
            theft['Location'] = theft['Location'].apply(address_to_latlong)
            m_theft['Location'] = m_theft['Location'].apply(address_to_latlong)
            
        theft.to_csv('./' + 'theft_' + file.split('Complete.csv')[0] + '.csv', sep=';', index = False)
        m_theft.to_csv('./' + 'm_theft_' + file.split('Complete.csv')[0] + '.csv', sep=';', index = False)
        print(f'Finished {file}')

Parsed: 2700 W ATKINSON AV, Milwaukee, WI - [43.0972881202978, -87.9469844363963]
Parsed: 4126 N TEUTONIA AV, Milwaukee, WI - [43.092067842605054, -87.9400393407874]
Parsed: 5180 N 32ND ST, Milwaukee, WI - [43.11141234307021, -87.95264235685009]
Parsed: 4125 N TEUTONIA AV, Milwaukee, WI - [43.0917851338613, -87.94006995278245]
Parsed: 5239 N 38TH ST, Milwaukee, WI - [43.11276286372555, -87.96006965753918]
Parsed: 4026 N 18TH ST, Milwaukee, WI - [43.090198826533566, -87.93315241185313]
Parsed: 6091 N TEUTONIA AV, Milwaukee, WI - [43.12798018436785, -87.95214425670802]
Parsed: 4519 N GREEN BAY AV, Milwaukee, WI - [43.09924205606212, -87.92858467949637]
Parsed: 3600 W ROHR AV, Milwaukee, WI - [43.113822277047944, -87.95767227704793]
Parsed: 4108 N 24TH PL, Milwaukee, WI - [43.09183822009348, -87.94341392627985]
Parsed: 4065-A N 24TH PL, Milwaukee, WI - [43.091198779906506, -87.94349862479058]
Parsed: 4305 N TEUTONIA AV, Milwaukee, WI - [43.095658222315194, -87.94213373581829]
Parsed: 5036

Parsed: 5500 W VLIET ST, Milwaukee, WI - [43.04951540940031, -87.9823779045242]
Parsed: 1319 N 46TH ST, Milwaukee, WI - [43.047864000000004, -87.97139806592986]
Parsed: 2812 N 62ND ST, Milwaukee, WI - [43.07009400000001, -87.98965335356932]
Parsed: 149 N 69TH ST, Milwaukee, WI - [43.03244683236193, -87.99867163200395]
Parsed: 7312 W APPLETON AV, Milwaukee, WI - [43.08880069276926, -88.00304722895163]
Parsed: 1 S BREWERS WA, Milwaukee, WI - [43.04200000000003, -87.90686999999997]
Parsed: 6330 W RICHMOND AV, Milwaukee, WI - [43.070826518754565, -87.99159141618097]
Parsed: 531 S 72ND ST, Milwaukee, WI - [43.02654274854291, -88.00240107372014]
Parsed: 837 N HAWLEY RD, Milwaukee, WI - [43.03998748893465, -87.9835268789654]
Parsed: 5301 W BURLEIGH ST, Milwaukee, WI - [43.075287525102546, -87.98003116763806]
Parsed: 2547 N 56TH ST, Milwaukee, WI - [43.065350025535224, -87.98304763200395]
Parsed: 435 N 91ST ST, Milwaukee, WI - [43.035567443245405, -88.02548724259896]
Parsed: 2409 N 52ND ST, Mi

Parsed: 1579 S 9TH ST, Milwaukee, WI - [43.01432077990651, -87.92266057703836]
Parsed: 541 W LAPHAM BL, Milwaukee, WI - [43.01403823388426, -87.91790690159327]
Parsed: 541 W LAPHAM BL, Milwaukee, WI - [43.01403823388426, -87.91790690159327]
Parsed: 1670 S 11TH ST, Milwaukee, WI - [43.01274574136568, -87.92544788992879]
Parsed: 106 W SEEBOTH ST, Milwaukee, WI - [43.03044146162459, -87.9114209403312]
Parsed: 117 W PITTSBURGH AV, Milwaukee, WI - [43.02937247013694, -87.91175249999999]
Parsed: 711 WEST PIERCE ST, Milwaukee, WI - [43.02410821436809, -87.91987567746936]
Parsed: 861 W WINDLAKE AV #UPR, Milwaukee, WI - [43.00957666108852, -87.92249205381175]
Parsed: 2344 S 16TH ST, Milwaukee, WI - [43.0019274970858, -87.93332639742641]
Parsed: 1400 W BECHER ST, Milwaukee, WI - [43.00657865503342, -87.92978062009875]
Parsed: 2527 S 15TH ST, Milwaukee, WI - [42.99861552844942, -87.93130009925535]
Parsed: 1819 S 2ND ST, Milwaukee, WI - [43.01086654484326, -87.91261783813874]
Parsed: 1501 S PEARL 

Parsed: 2993-A S DELAWARE AV, Milwaukee, WI - [42.98927467164879, -87.87959619054756]
Parsed: 401 E WARD ST, Milwaukee, WI - [43.00510848866055, -87.9044747251596]
Parsed: 381 E WARD ST, Milwaukee, WI - [43.0045767512631, -87.9053659826725]
Parsed: 2345 S HOWELL AV, Milwaukee, WI - [43.00191058673323, -87.90479806318855]
Parsed: 2950 S CHASE AV, Milwaukee, WI - [42.99098520961922, -87.90945266322424]
Parsed: 3912 S HOWELL AV, Milwaukee, WI - [42.97325173688614, -87.90939509002278]
Parsed: 2691 S KINNICKINNIC AV, Milwaukee, WI - [42.99607446015362, -87.89699490342768]
Parsed: 2248 S CHASE AV, Milwaukee, WI - [43.00350471801601, -87.91858218676799]
Parsed: 3201 S BURRELL ST, Milwaukee, WI - [42.98634958673321, -87.90819004818492]
Parsed: 2200 S 4TH ST, Milwaukee, WI - [43.004527742714515, -87.91544493349322]
Parsed: 270 W HOLT AV, Milwaukee, WI - [42.982445522649684, -87.91377299999999]
Parsed: 2204 S KINNICKINNIC AV, Milwaukee, WI - [43.004475241284055, -87.90543319005573]
Parsed: 2916 

Parsed: 2226 N 44TH ST, Milwaukee, WI - [43.059887748542906, -87.96882488631792]
Parsed: 1834 W WALNUT ST, Milwaukee, WI - [43.052712504327836, -87.93619272300768]
Parsed: 2742 N 15TH ST, Milwaukee, WI - [43.06868916180969, -87.93122641185313]
Parsed: 1811 W CENTER ST, Milwaukee, WI - [43.067765495672184, -87.93540394463038]
Parsed: 2210 W HADLEY ST, Milwaukee, WI - [43.06971458496542, -87.94008888356565]
Parsed: 1700 W CENTER ST, Milwaukee, WI - [43.067851460895504, -87.93450043832739]
Parsed: 1909 W CLARKE ST, Milwaukee, WI - [43.06590450678069, -87.93667391909516]
Parsed: 2612 N 45TH ST, Milwaukee, WI - [43.06643941326678, -87.96993286078269]
Parsed: 2611 N 39TH ST, Milwaukee, WI - [43.06626033527613, -87.96222007703837]
Parsed: 2328 N 39TH ST, Milwaukee, WI - [43.06109769515457, -87.96222565213085]
Parsed: 2449 N 36TH ST, Milwaukee, WI - [43.06323641909515, -87.95868807372014]
Parsed: 2428 N 39TH ST #LOWER, Milwaukee, WI - [43.06313599417163, -87.9622164190665]
Parsed: 1300 W FOND 

...

Parsed: 5339 N 39TH ST, Milwaukee, WI - [43.11462717346646, -87.96123057372014]
Parsed: 4159 N 15TH ST, Milwaukee, WI - [43.092765366639725, -87.92945059925536]
Parsed: 2128 W ATKINSON AV, Milwaukee, WI - [43.09178856676657, -87.93905513347583]
Parsed: 4035 N 19TH PL, Milwaukee, WI - [43.090277188271216, -87.93572574948107]
Parsed: 4722 N 31ST ST, Milwaukee, WI - [43.10304266472389, -87.95164635356933]
Parsed: 4456 N TEUTONIA AV, Milwaukee, WI - [43.097625573171875, -87.9430614684931]
Parsed: 4349 N 14TH ST, Milwaukee, WI - [43.09605053427779, -87.92811859925536]
Parsed: 5131 N GREEN BAY AV, Milwaukee, WI - [43.11047604535722, -87.93284446503287]
Parsed: 3118 W CAMERON AV, Milwaukee, WI - [43.107329460470744, -87.95229372300767]
Parsed: 4856 N 22ND ST, Milwaukee, WI - [43.10534867028629, -87.93920963442478]
Parsed: 4102 N 27TH ST, Milwaukee, WI - [43.09132909358355, -87.94687730078597]
Parsed: 4740 W HAMPTON AV, Milwaukee, WI - [43.10473049321934, -87.9726015]
Parsed: 5445 N 27TH ST, M

Parsed: 1501 W MITCHELL ST, Milwaukee, WI - [43.01225946681874, -87.93106183236193]
Parsed: 1509 S MUSKEGO AV, Milwaukee, WI - [43.015709952308995, -87.93452487929268]
Parsed: 1521 W ROGERS ST, Milwaukee, WI - [43.00826170547654, -87.93151021649165]
Parsed: 1815 S 12TH ST, Milwaukee, WI - [43.01063044463038, -87.92700904818491]
Parsed: 1313 S 6TH ST, Milwaukee, WI - [43.01756096863639, -87.9182775337582]
Parsed: 1332 S 4TH ST, Milwaukee, WI - [43.017248263113856, -87.91532301788916]
Parsed: 2146 S 15TH ST, Milwaukee, WI - [43.00545554954124, -87.93106248456367]
Parsed: 605 S 1ST ST, Milwaukee, WI - [43.02514958612625, -87.9111919060777]
Parsed: 1935 S 11TH ST, Milwaukee, WI - [43.00932776019968, -87.92565801154119]
Parsed: 2054 S 15TH PL, Milwaukee, WI - [43.00706613044608, -87.93224503173897]
Parsed: 1670 S 11TH ST, Milwaukee, WI - [43.01274574136568, -87.92544788992879]
Parsed: 1905 S 6TH ST, Milwaukee, WI - [43.00989358673323, -87.91865059925536]
Parsed: 1020 W HISTORIC MITCHELL ST,

Parsed: 2430 N SHERMAN BL, Milwaukee, WI - [43.06313641326679, -87.96740591517137]
Parsed: 1350 W RESERVOIR AV, Milwaukee, WI - [43.05576149321934, -87.92988241326678]
Parsed: 2437 N 45TH ST, Milwaukee, WI - [43.06325433527613, -87.97002807372014]
Parsed: 4101 W WRIGHT ST, Milwaukee, WI - [43.06414778598241, -87.96464271401759]
Parsed: 2467 W WALNUT ST, Milwaukee, WI - [43.05270953231589, -87.94453066763806]
Parsed: 2300 W WALNUT ST, Milwaukee, WI - [43.052811199411295, -87.9414946994113]
Parsed: 2766 N 24TH PL, Milwaukee, WI - [43.06936789203073, -87.94359444460173]
Parsed: 2451 N 25TH ST, Milwaukee, WI - [43.06349719900166, -87.944968092042]
Parsed: 5718 N 75TH ST, Milwaukee, WI - [43.12159124562871, -88.00463588631791]
Parsed: 6353 N JOYCE AV, Milwaukee, WI - [43.13321484540368, -88.01942908912781]
Parsed: 6414 W SHERIDAN AV, Milwaukee, WI - [43.11848796139836, -87.99126885425297]
Parsed: 8804 W VILLARD AV, Milwaukee, WI - [43.112292275098035, -88.02228606202056]
Parsed: 5523 N 91ST

Parsed: 941 N JAMES LOVELL ST, Milwaukee, WI - [43.04239264536553, -87.92034795356999]
Parsed: 1506 N 34TH ST, Milwaukee, WI - [43.05033900000001, -87.9564159190665]
Parsed: 1145 N CALLAHAN PL, Milwaukee, WI - [43.04510516846553, -87.93233601127733]
Parsed: 902 W WELLS ST, Milwaukee, WI - [43.040311553356084, -87.92315817906237]
Parsed: 929 N 17TH ST, Milwaukee, WI - [43.04232933527612, -87.93415857372014]
Parsed: 2035 W GALENA ST, Milwaukee, WI - [43.051295802315266, -87.93814542587614]
Parsed: 3223 W MC KINLEY BL, Milwaukee, WI - [43.04721754284748, -87.95421008381904]
Parsed: 1331 N 21ST ST, Milwaukee, WI - [43.047699890112334, -87.93907084800104]
Parsed: 2308 W CHERRY ST, Milwaukee, WI - [43.05006546768412, -87.94187983236193]
Parsed: 1122 N 22ND ST, Milwaukee, WI - [43.04476581848868, -87.94027145876773]
Parsed: 1117 N 14TH ST, Milwaukee, WI - [43.044849, -87.93014907372014]
Parsed: 3006 W HIGHLAND BL, Milwaukee, WI - [43.044640457152525, -87.95199141618096]
Parsed: 2711 W JUNEAU 

Parsed: 3200 W LOCUST ST, Milwaukee, WI - [43.07162181172015, -87.95400862658151]
Parsed: 2943 N 26TH ST, Milwaukee, WI - [43.07239769608748, -87.94607909536022]
Parsed: 3608 N 50TH ST, Milwaukee, WI - [43.08367399417162, -87.9758458607827]
Parsed: 4467 N 55TH ST, Milwaukee, WI - [43.09869586372554, -87.98183810646873]
Parsed: 3041 N 48TH ST, Milwaukee, WI - [43.07427033527614, -87.97361963532217]
Parsed: 4760 N 42ND ST, Milwaukee, WI - [43.103880000000004, -87.9649853280341]
Parsed: 3233 W BURLEIGH ST, Milwaukee, WI - [43.07518247013696, -87.954822]
Parsed: 3177 N 48TH ST, Milwaukee, WI - [43.07689841909516, -87.97378107372015]
Parsed: 3626 W FOND DU LAC AV, Milwaukee, WI - [43.07692945158396, -87.95944045848019]
Parsed: 4029 N 42ND ST, Milwaukee, WI - [43.09054308964741, -87.96533807761527]
Parsed: 3509 W HAMPTON AV, Milwaukee, WI - [43.104602495672175, -87.95709]
Parsed: 5829 W VILLARD AV, Milwaukee, WI - [43.111967481245465, -87.98579983236193]
Parsed: 3009 N 37TH ST, Milwaukee, WI